In [1]:
# =========================
# Count-only K-auto (Multi-event) + Windowing version
#
# ✅ Windowing added:
# - TRAIN: trial -> sliding windows (window-level count = trial-average rate * window duration)
# - TEST : trial 그대로 두고, windowing inference로 window rate 평균 -> 전체 count 예측
# - k_hat / entropy / rep_rate / phase heatmap은 (표현학습 확인용) full-trial 1회 forward로 기록
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (표준화) 평균=0, std=1
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T, C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) ✅ Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows 확장
    window 라벨은 trial-level 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> sliding windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (이미 정규화된 상태)
    return: pred_count(float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)  # (B,T,K), sum=1
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    """
    - outputs K_max micro-event rates r_k(t)
    - predicts k_hat (>=1) per sample
    - rep_rate(t) = micro_rate(t) / k_hat
    """
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp logit bias만 -2

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        # ✅ BUGFIX: use tau argument (was tau=1.0 fixed)
        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p  # (B,T,K)

        micro_rate_t = amp_t  # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,
            "phase_p": phase_p,
            "phase_logits": phase_logits,
            "micro_rate_t": micro_rate_t,
            "rep_rate_t": rep_rate_t,
            "k_hat": k_hat,
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps  # valid(B*T)*C
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])  # (B,T-1)
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)  # (B,K)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)  # (B,T,1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)         # (B,C,T)
        mask = batch["mask"].to(device)      # (B,T)
        y_count = batch["count"].to(device)  # (B,)
        length = batch["length"].to(device)  # (B,)

        duration = torch.clamp(length / fs, min=1e-6)  # sec
        y_rate = y_count / duration                    # reps/s

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = len(loader)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers (subject-wise subplot)
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)  # (T,)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(
    phase_p_np,
    fs,
    title="phase_p heatmap + dominant phase",
    max_T=2000
):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)  # (T',K)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)  # (T',)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :],
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization (only test_subj)"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred(win) {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main (LOSO)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
            12: 'Jump front & back'
        },
        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ]
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # ✅ Windowing Params (added)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        # temperature
        "tau": 1.0,

        # Count-only labels
        "ALL_LABELS": [
            ("subject1", 6, 21),
            ("subject2", 6, 19),
            ("subject3", 6, 21),
            ("subject4", 6, 20),
            ("subject5", 6, 20),
            ("subject6", 6, 20),
            ("subject7", 6, 20),
            ("subject8", 6, 21),
            ("subject9", 6, 21),
            ("subject10", 6, 20),
        ],
    }

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    loso_results = []

    print("\n" + "-"*80)
    print(" >>> Starting LOSO (count-only, K-auto) + WINDOWING")
    print("-"*80)

    viz_cache = []

    for fold_idx, test_subj in enumerate(subjects):
        set_strict_seed(CONFIG["seed"])

        train_labels = [x for x in CONFIG["ALL_LABELS"] if x[0] != test_subj]
        test_labels  = [x for x in CONFIG["ALL_LABELS"] if x[0] == test_subj]

        # ---- trial-level ----
        train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if not test_trials:
            print(f"[Skip] Fold {fold_idx+1}: {test_subj} has no data.")
            continue

        # ✅ TRAIN only: trial -> windows
        train_data = trial_list_to_windows(
            train_trials,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            drop_last=CONFIG["drop_last"]
        )

        # TEST: trial 그대로
        test_data = test_trials

        g = torch.Generator()
        g.manual_seed(CONFIG["seed"])

        train_loader = DataLoader(
            TrialDataset(train_data),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            generator=g,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        model.eval()

        # ---- fold test (trial 단위) ----
        fold_mae = 0.0
        fold_res_str = ""

        test_gt = None
        test_pred = None
        test_diff = None
        test_khat = None
        test_entropy = None

        test_t = None
        test_rep_rate = None
        test_phase_p = None

        for item in test_data:
            x_np = item["data"]  # (T,C)
            T = x_np.shape[0]

            # ✅ pred: windowing inference
            count_pred_win, _win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )

            count_gt = float(item["count"])
            abs_err = abs(count_pred_win - count_gt)
            fold_mae += abs_err
            fold_res_str += f"[Pred(win): {count_pred_win:.1f} / GT: {count_gt:.0f}]"

            # ✅ 표현학습 확인용: full-trial 1회 forward (k_hat/entropy/rep_rate/phase)
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            test_gt = count_gt
            test_pred = float(count_pred_win)
            test_diff = float(count_pred_win - count_gt)
            test_khat = k_hat
            test_entropy = ent

            rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()  # (T,)
            test_rep_rate = rep_rate
            test_t = np.arange(T) / CONFIG["fs"]
            test_phase_p = phase_p

        fold_mae /= len(test_data)
        loso_results.append(fold_mae)

        print(f"Fold {fold_idx+1:2d} | Test: {test_subj} | MAE: {fold_mae:.2f} | {fold_res_str}")

        if (test_gt is not None) and (test_pred is not None):
            print(
                f"[Fold TEST Summary] {test_subj} | GT={test_gt:.0f} | Pred(win)={test_pred:.2f} | "
                f"Diff={test_diff:+.2f} | k_hat(full)={test_khat:.2f} | phase_entropy(full)={test_entropy:.3f}"
            )

        if (test_t is not None) and (test_rep_rate is not None):
            viz_cache.append({
                "fold": fold_idx + 1,
                "test_subj": test_subj,
                "t": test_t,
                "rep_rate": test_rep_rate,
                "gt": float(test_gt) if test_gt is not None else 0.0,
                "pred": float(test_pred) if test_pred is not None else 0.0,
                "diff": float(test_diff) if test_diff is not None else 0.0,
                "k_hat": float(test_khat) if test_khat is not None else 0.0,
                "entropy": float(test_entropy) if test_entropy is not None else 0.0,
                "phase_p": test_phase_p,
            })

    print("-"*80)
    print(f" >>> Final LOSO Result (Average MAE): {np.mean(loso_results):.3f}")
    print(f" >>> Standard Deviation: {np.std(loso_results):.3f}")
    print("-"*80)

    # plot_folds_test_subplot(
    #     viz_cache,
    #     fs=CONFIG["fs"],
    #     title="Fold-wise TEST visualization (Pred by window-avg rate)"
    # )

    # for item in viz_cache:
    #     plot_phase_heatmap_and_dominant(
    #         item["phase_p"],
    #         fs=CONFIG["fs"],
    #         title=f"[Fold {item['fold']:2d}] {item['test_subj']} | k_hat(full)={item['k_hat']:.2f} | ent(full)={item['entropy']:.3f}",
    #         max_T=2000
    #     )


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

--------------------------------------------------------------------------------
 >>> Starting LOSO (count-only, K-auto) + WINDOWING
--------------------------------------------------------------------------------
Fold  1 | Test: subject1 | MAE: 1.39 | [Pred(win): 19.6 / GT: 21]
[Fold TEST Summary] subject1 | GT=21 | Pred(win)=19.61 | Diff=-1.39 | k_hat(full)=1.02 | phase_entropy(full)=0.038
Fold  2 | Test: subject2 | MAE: 6.36 | [Pred(win): 25.4 / GT: 19]
[Fold TEST Summary] subject2 | GT=19 | Pred(win)=25.36 | Diff=+6.36 | k_hat(full)=1.06 | phase_entropy(full)=0.119
Fold  3 | Test: subject3 | MAE: 6.60 | [Pred(win): 27.6 / GT: 21]
[Fold TEST Summary] subject3 | GT=21 | Pred(win)=27.60 | Diff=+6.60 | k_hat(full)=1.01 | phase_entropy(full)=0.023
Fold  4 | Test: subject4 | MAE: 2.46 | [Pred(win): 22.5 / GT: 20]
[Fold TEST Summary] subject4 | GT=20 | Pred(win)=22.46 | Diff=+2.46 | k_

In [2]:
# =========================
# Count-only K-auto (Multi-event) + Windowing version
#
# ✅ Windowing added:
# - TRAIN: trial -> sliding windows (window-level count = trial-average rate * window duration)
# - TEST : trial 그대로 두고, windowing inference로 window rate 평균 -> 전체 count 예측
# - k_hat / entropy / rep_rate / phase heatmap은 (표현학습 확인용) full-trial 1회 forward로 기록
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (표준화) 평균=0, std=1
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T, C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) ✅ Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows 확장
    window 라벨은 trial-level 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> sliding windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (이미 정규화된 상태)
    return: pred_count(float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)  # (B,T,K), sum=1
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    """
    - outputs K_max micro-event rates r_k(t)
    - predicts k_hat (>=1) per sample
    - rep_rate(t) = micro_rate(t) / k_hat
    """
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp logit bias만 -2

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        # ✅ BUGFIX: use tau argument (was tau=1.0 fixed)
        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p  # (B,T,K)

        micro_rate_t = amp_t  # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,
            "phase_p": phase_p,
            "phase_logits": phase_logits,
            "micro_rate_t": micro_rate_t,
            "rep_rate_t": rep_rate_t,
            "k_hat": k_hat,
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps  # valid(B*T)*C
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])  # (B,T-1)
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)  # (B,K)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)  # (B,T,1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)         # (B,C,T)
        mask = batch["mask"].to(device)      # (B,T)
        y_count = batch["count"].to(device)  # (B,)
        length = batch["length"].to(device)  # (B,)

        duration = torch.clamp(length / fs, min=1e-6)  # sec
        y_rate = y_count / duration                    # reps/s

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = len(loader)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers (subject-wise subplot)
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)  # (T,)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(
    phase_p_np,
    fs,
    title="phase_p heatmap + dominant phase",
    max_T=2000
):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)  # (T',K)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)  # (T',)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :],
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization (only test_subj)"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred(win) {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main (LOSO)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
            12: 'Jump front & back'
        },
        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ]
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # ✅ Windowing Params (added)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        # temperature
        "tau": 1.0,

        # Count-only labels
        "ALL_LABELS": [
            ("subject1", 7, 20),
            ("subject2", 7, 20),
            ("subject3", 7, 20),
            ("subject4", 7, 20),
            ("subject5", 7, 20),
            ("subject6", 7, 20),
            ("subject7", 7, 20),
            ("subject8", 7, 19),
            ("subject9", 7, 19),
            ("subject10", 7, 20),
        ],
    }

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    loso_results = []

    print("\n" + "-"*80)
    print(" >>> Starting LOSO (count-only, K-auto) + WINDOWING")
    print("-"*80)

    viz_cache = []

    for fold_idx, test_subj in enumerate(subjects):
        set_strict_seed(CONFIG["seed"])

        train_labels = [x for x in CONFIG["ALL_LABELS"] if x[0] != test_subj]
        test_labels  = [x for x in CONFIG["ALL_LABELS"] if x[0] == test_subj]

        # ---- trial-level ----
        train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if not test_trials:
            print(f"[Skip] Fold {fold_idx+1}: {test_subj} has no data.")
            continue

        # ✅ TRAIN only: trial -> windows
        train_data = trial_list_to_windows(
            train_trials,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            drop_last=CONFIG["drop_last"]
        )

        # TEST: trial 그대로
        test_data = test_trials

        g = torch.Generator()
        g.manual_seed(CONFIG["seed"])

        train_loader = DataLoader(
            TrialDataset(train_data),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            generator=g,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        model.eval()

        # ---- fold test (trial 단위) ----
        fold_mae = 0.0
        fold_res_str = ""

        test_gt = None
        test_pred = None
        test_diff = None
        test_khat = None
        test_entropy = None

        test_t = None
        test_rep_rate = None
        test_phase_p = None

        for item in test_data:
            x_np = item["data"]  # (T,C)
            T = x_np.shape[0]

            # ✅ pred: windowing inference
            count_pred_win, _win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )

            count_gt = float(item["count"])
            abs_err = abs(count_pred_win - count_gt)
            fold_mae += abs_err
            fold_res_str += f"[Pred(win): {count_pred_win:.1f} / GT: {count_gt:.0f}]"

            # ✅ 표현학습 확인용: full-trial 1회 forward (k_hat/entropy/rep_rate/phase)
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            test_gt = count_gt
            test_pred = float(count_pred_win)
            test_diff = float(count_pred_win - count_gt)
            test_khat = k_hat
            test_entropy = ent

            rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()  # (T,)
            test_rep_rate = rep_rate
            test_t = np.arange(T) / CONFIG["fs"]
            test_phase_p = phase_p

        fold_mae /= len(test_data)
        loso_results.append(fold_mae)

        print(f"Fold {fold_idx+1:2d} | Test: {test_subj} | MAE: {fold_mae:.2f} | {fold_res_str}")

        if (test_gt is not None) and (test_pred is not None):
            print(
                f"[Fold TEST Summary] {test_subj} | GT={test_gt:.0f} | Pred(win)={test_pred:.2f} | "
                f"Diff={test_diff:+.2f} | k_hat(full)={test_khat:.2f} | phase_entropy(full)={test_entropy:.3f}"
            )

        if (test_t is not None) and (test_rep_rate is not None):
            viz_cache.append({
                "fold": fold_idx + 1,
                "test_subj": test_subj,
                "t": test_t,
                "rep_rate": test_rep_rate,
                "gt": float(test_gt) if test_gt is not None else 0.0,
                "pred": float(test_pred) if test_pred is not None else 0.0,
                "diff": float(test_diff) if test_diff is not None else 0.0,
                "k_hat": float(test_khat) if test_khat is not None else 0.0,
                "entropy": float(test_entropy) if test_entropy is not None else 0.0,
                "phase_p": test_phase_p,
            })

    print("-"*80)
    print(f" >>> Final LOSO Result (Average MAE): {np.mean(loso_results):.3f}")
    print(f" >>> Standard Deviation: {np.std(loso_results):.3f}")
    print("-"*80)

    # plot_folds_test_subplot(
    #     viz_cache,
    #     fs=CONFIG["fs"],
    #     title="Fold-wise TEST visualization (Pred by window-avg rate)"
    # )

    # for item in viz_cache:
    #     plot_phase_heatmap_and_dominant(
    #         item["phase_p"],
    #         fs=CONFIG["fs"],
    #         title=f"[Fold {item['fold']:2d}] {item['test_subj']} | k_hat(full)={item['k_hat']:.2f} | ent(full)={item['entropy']:.3f}",
    #         max_T=2000
    #     )


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

--------------------------------------------------------------------------------
 >>> Starting LOSO (count-only, K-auto) + WINDOWING
--------------------------------------------------------------------------------
Fold  1 | Test: subject1 | MAE: 0.09 | [Pred(win): 19.9 / GT: 20]
[Fold TEST Summary] subject1 | GT=20 | Pred(win)=19.91 | Diff=-0.09 | k_hat(full)=1.01 | phase_entropy(full)=0.028
Fold  2 | Test: subject2 | MAE: 2.63 | [Pred(win): 22.6 / GT: 20]
[Fold TEST Summary] subject2 | GT=20 | Pred(win)=22.63 | Diff=+2.63 | k_hat(full)=1.01 | phase_entropy(full)=0.020
Fold  3 | Test: subject3 | MAE: 4.18 | [Pred(win): 24.2 / GT: 20]
[Fold TEST Summary] subject3 | GT=20 | Pred(win)=24.18 | Diff=+4.18 | k_hat(full)=1.01 | phase_entropy(full)=0.026
Fold  4 | Test: subject4 | MAE: 1.62 | [Pred(win): 21.6 / GT: 20]
[Fold TEST Summary] subject4 | GT=20 | Pred(win)=21.62 | Diff=+1.62 | k_

In [3]:
# =========================
# Count-only K-auto (Multi-event) + Windowing version
#
# ✅ Windowing added:
# - TRAIN: trial -> sliding windows (window-level count = trial-average rate * window duration)
# - TEST : trial 그대로 두고, windowing inference로 window rate 평균 -> 전체 count 예측
# - k_hat / entropy / rep_rate / phase heatmap은 (표현학습 확인용) full-trial 1회 forward로 기록
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (표준화) 평균=0, std=1
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T, C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) ✅ Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows 확장
    window 라벨은 trial-level 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> sliding windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (이미 정규화된 상태)
    return: pred_count(float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)  # (B,T,K), sum=1
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    """
    - outputs K_max micro-event rates r_k(t)
    - predicts k_hat (>=1) per sample
    - rep_rate(t) = micro_rate(t) / k_hat
    """
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp logit bias만 -2

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        # ✅ BUGFIX: use tau argument (was tau=1.0 fixed)
        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p  # (B,T,K)

        micro_rate_t = amp_t  # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,
            "phase_p": phase_p,
            "phase_logits": phase_logits,
            "micro_rate_t": micro_rate_t,
            "rep_rate_t": rep_rate_t,
            "k_hat": k_hat,
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps  # valid(B*T)*C
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])  # (B,T-1)
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)  # (B,K)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)  # (B,T,1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)         # (B,C,T)
        mask = batch["mask"].to(device)      # (B,T)
        y_count = batch["count"].to(device)  # (B,)
        length = batch["length"].to(device)  # (B,)

        duration = torch.clamp(length / fs, min=1e-6)  # sec
        y_rate = y_count / duration                    # reps/s

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = len(loader)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers (subject-wise subplot)
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)  # (T,)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(
    phase_p_np,
    fs,
    title="phase_p heatmap + dominant phase",
    max_T=2000
):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)  # (T',K)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)  # (T',)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :],
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization (only test_subj)"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred(win) {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main (LOSO)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
            12: 'Jump front & back'
        },
        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ]
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # ✅ Windowing Params (added)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        # temperature
        "tau": 1.0,

        # Count-only labels
        "ALL_LABELS": [
            ("subject1", 8, 20),
            ("subject2", 8, 21),
            ("subject3", 8, 21),
            ("subject4", 8, 19),
            ("subject5", 8, 20),
            ("subject6", 8, 20),
            ("subject7", 8, 21),
            ("subject8", 8, 21),
            ("subject9", 8, 21),
            ("subject10", 8, 21),
        ],
    }

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    loso_results = []

    print("\n" + "-"*80)
    print(" >>> Starting LOSO (count-only, K-auto) + WINDOWING")
    print("-"*80)

    viz_cache = []

    for fold_idx, test_subj in enumerate(subjects):
        set_strict_seed(CONFIG["seed"])

        train_labels = [x for x in CONFIG["ALL_LABELS"] if x[0] != test_subj]
        test_labels  = [x for x in CONFIG["ALL_LABELS"] if x[0] == test_subj]

        # ---- trial-level ----
        train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if not test_trials:
            print(f"[Skip] Fold {fold_idx+1}: {test_subj} has no data.")
            continue

        # ✅ TRAIN only: trial -> windows
        train_data = trial_list_to_windows(
            train_trials,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            drop_last=CONFIG["drop_last"]
        )

        # TEST: trial 그대로
        test_data = test_trials

        g = torch.Generator()
        g.manual_seed(CONFIG["seed"])

        train_loader = DataLoader(
            TrialDataset(train_data),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            generator=g,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        model.eval()

        # ---- fold test (trial 단위) ----
        fold_mae = 0.0
        fold_res_str = ""

        test_gt = None
        test_pred = None
        test_diff = None
        test_khat = None
        test_entropy = None

        test_t = None
        test_rep_rate = None
        test_phase_p = None

        for item in test_data:
            x_np = item["data"]  # (T,C)
            T = x_np.shape[0]

            # ✅ pred: windowing inference
            count_pred_win, _win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )

            count_gt = float(item["count"])
            abs_err = abs(count_pred_win - count_gt)
            fold_mae += abs_err
            fold_res_str += f"[Pred(win): {count_pred_win:.1f} / GT: {count_gt:.0f}]"

            # ✅ 표현학습 확인용: full-trial 1회 forward (k_hat/entropy/rep_rate/phase)
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            test_gt = count_gt
            test_pred = float(count_pred_win)
            test_diff = float(count_pred_win - count_gt)
            test_khat = k_hat
            test_entropy = ent

            rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()  # (T,)
            test_rep_rate = rep_rate
            test_t = np.arange(T) / CONFIG["fs"]
            test_phase_p = phase_p

        fold_mae /= len(test_data)
        loso_results.append(fold_mae)

        print(f"Fold {fold_idx+1:2d} | Test: {test_subj} | MAE: {fold_mae:.2f} | {fold_res_str}")

        if (test_gt is not None) and (test_pred is not None):
            print(
                f"[Fold TEST Summary] {test_subj} | GT={test_gt:.0f} | Pred(win)={test_pred:.2f} | "
                f"Diff={test_diff:+.2f} | k_hat(full)={test_khat:.2f} | phase_entropy(full)={test_entropy:.3f}"
            )

        if (test_t is not None) and (test_rep_rate is not None):
            viz_cache.append({
                "fold": fold_idx + 1,
                "test_subj": test_subj,
                "t": test_t,
                "rep_rate": test_rep_rate,
                "gt": float(test_gt) if test_gt is not None else 0.0,
                "pred": float(test_pred) if test_pred is not None else 0.0,
                "diff": float(test_diff) if test_diff is not None else 0.0,
                "k_hat": float(test_khat) if test_khat is not None else 0.0,
                "entropy": float(test_entropy) if test_entropy is not None else 0.0,
                "phase_p": test_phase_p,
            })

    print("-"*80)
    print(f" >>> Final LOSO Result (Average MAE): {np.mean(loso_results):.3f}")
    print(f" >>> Standard Deviation: {np.std(loso_results):.3f}")
    print("-"*80)

    # plot_folds_test_subplot(
    #     viz_cache,
    #     fs=CONFIG["fs"],
    #     title="Fold-wise TEST visualization (Pred by window-avg rate)"
    # )

    # for item in viz_cache:
    #     plot_phase_heatmap_and_dominant(
    #         item["phase_p"],
    #         fs=CONFIG["fs"],
    #         title=f"[Fold {item['fold']:2d}] {item['test_subj']} | k_hat(full)={item['k_hat']:.2f} | ent(full)={item['entropy']:.3f}",
    #         max_T=2000
    #     )


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

--------------------------------------------------------------------------------
 >>> Starting LOSO (count-only, K-auto) + WINDOWING
--------------------------------------------------------------------------------
Fold  1 | Test: subject1 | MAE: 12.18 | [Pred(win): 32.2 / GT: 20]
[Fold TEST Summary] subject1 | GT=20 | Pred(win)=32.18 | Diff=+12.18 | k_hat(full)=1.04 | phase_entropy(full)=0.084
Fold  2 | Test: subject2 | MAE: 1.54 | [Pred(win): 22.5 / GT: 21]
[Fold TEST Summary] subject2 | GT=21 | Pred(win)=22.54 | Diff=+1.54 | k_hat(full)=1.05 | phase_entropy(full)=0.103
Fold  3 | Test: subject3 | MAE: 4.99 | [Pred(win): 16.0 / GT: 21]
[Fold TEST Summary] subject3 | GT=21 | Pred(win)=16.01 | Diff=-4.99 | k_hat(full)=1.04 | phase_entropy(full)=0.085
Fold  4 | Test: subject4 | MAE: 5.68 | [Pred(win): 24.7 / GT: 19]
[Fold TEST Summary] subject4 | GT=19 | Pred(win)=24.68 | Diff=+5.68 | 

In [4]:
# =========================
# Count-only K-auto (Multi-event) + Windowing version
#
# ✅ Windowing added:
# - TRAIN: trial -> sliding windows (window-level count = trial-average rate * window duration)
# - TEST : trial 그대로 두고, windowing inference로 window rate 평균 -> 전체 count 예측
# - k_hat / entropy / rep_rate / phase heatmap은 (표현학습 확인용) full-trial 1회 forward로 기록
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (표준화) 평균=0, std=1
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T, C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) ✅ Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows 확장
    window 라벨은 trial-level 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> sliding windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (이미 정규화된 상태)
    return: pred_count(float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)  # (B,T,K), sum=1
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    """
    - outputs K_max micro-event rates r_k(t)
    - predicts k_hat (>=1) per sample
    - rep_rate(t) = micro_rate(t) / k_hat
    """
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp logit bias만 -2

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        # ✅ BUGFIX: use tau argument (was tau=1.0 fixed)
        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p  # (B,T,K)

        micro_rate_t = amp_t  # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,
            "phase_p": phase_p,
            "phase_logits": phase_logits,
            "micro_rate_t": micro_rate_t,
            "rep_rate_t": rep_rate_t,
            "k_hat": k_hat,
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps  # valid(B*T)*C
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])  # (B,T-1)
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)  # (B,K)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)  # (B,T,1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)         # (B,C,T)
        mask = batch["mask"].to(device)      # (B,T)
        y_count = batch["count"].to(device)  # (B,)
        length = batch["length"].to(device)  # (B,)

        duration = torch.clamp(length / fs, min=1e-6)  # sec
        y_rate = y_count / duration                    # reps/s

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = len(loader)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers (subject-wise subplot)
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)  # (T,)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(
    phase_p_np,
    fs,
    title="phase_p heatmap + dominant phase",
    max_T=2000
):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)  # (T',K)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)  # (T',)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :],
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization (only test_subj)"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred(win) {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main (LOSO)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
            12: 'Jump front & back'
        },
        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ]
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # ✅ Windowing Params (added)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        # temperature
        "tau": 1.0,

        # Count-only labels
        "ALL_LABELS": [
            ("subject1", 12, 20),
            ("subject2", 12, 22),
            ("subject3", 12, 21),
            ("subject4", 12, 21),
            ("subject5", 12, 20),
            ("subject6", 12, 21),
            ("subject7", 12, 19),
            ("subject8", 12, 20),
            ("subject9", 12, 20),
            ("subject10", 12, 20),
        ],
    }

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    loso_results = []

    print("\n" + "-"*80)
    print(" >>> Starting LOSO (count-only, K-auto) + WINDOWING")
    print("-"*80)

    viz_cache = []

    for fold_idx, test_subj in enumerate(subjects):
        set_strict_seed(CONFIG["seed"])

        train_labels = [x for x in CONFIG["ALL_LABELS"] if x[0] != test_subj]
        test_labels  = [x for x in CONFIG["ALL_LABELS"] if x[0] == test_subj]

        # ---- trial-level ----
        train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if not test_trials:
            print(f"[Skip] Fold {fold_idx+1}: {test_subj} has no data.")
            continue

        # ✅ TRAIN only: trial -> windows
        train_data = trial_list_to_windows(
            train_trials,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            drop_last=CONFIG["drop_last"]
        )

        # TEST: trial 그대로
        test_data = test_trials

        g = torch.Generator()
        g.manual_seed(CONFIG["seed"])

        train_loader = DataLoader(
            TrialDataset(train_data),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            generator=g,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        model.eval()

        # ---- fold test (trial 단위) ----
        fold_mae = 0.0
        fold_res_str = ""

        test_gt = None
        test_pred = None
        test_diff = None
        test_khat = None
        test_entropy = None

        test_t = None
        test_rep_rate = None
        test_phase_p = None

        for item in test_data:
            x_np = item["data"]  # (T,C)
            T = x_np.shape[0]

            # ✅ pred: windowing inference
            count_pred_win, _win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )

            count_gt = float(item["count"])
            abs_err = abs(count_pred_win - count_gt)
            fold_mae += abs_err
            fold_res_str += f"[Pred(win): {count_pred_win:.1f} / GT: {count_gt:.0f}]"

            # ✅ 표현학습 확인용: full-trial 1회 forward (k_hat/entropy/rep_rate/phase)
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            test_gt = count_gt
            test_pred = float(count_pred_win)
            test_diff = float(count_pred_win - count_gt)
            test_khat = k_hat
            test_entropy = ent

            rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()  # (T,)
            test_rep_rate = rep_rate
            test_t = np.arange(T) / CONFIG["fs"]
            test_phase_p = phase_p

        fold_mae /= len(test_data)
        loso_results.append(fold_mae)

        print(f"Fold {fold_idx+1:2d} | Test: {test_subj} | MAE: {fold_mae:.2f} | {fold_res_str}")

        if (test_gt is not None) and (test_pred is not None):
            print(
                f"[Fold TEST Summary] {test_subj} | GT={test_gt:.0f} | Pred(win)={test_pred:.2f} | "
                f"Diff={test_diff:+.2f} | k_hat(full)={test_khat:.2f} | phase_entropy(full)={test_entropy:.3f}"
            )

        if (test_t is not None) and (test_rep_rate is not None):
            viz_cache.append({
                "fold": fold_idx + 1,
                "test_subj": test_subj,
                "t": test_t,
                "rep_rate": test_rep_rate,
                "gt": float(test_gt) if test_gt is not None else 0.0,
                "pred": float(test_pred) if test_pred is not None else 0.0,
                "diff": float(test_diff) if test_diff is not None else 0.0,
                "k_hat": float(test_khat) if test_khat is not None else 0.0,
                "entropy": float(test_entropy) if test_entropy is not None else 0.0,
                "phase_p": test_phase_p,
            })

    print("-"*80)
    print(f" >>> Final LOSO Result (Average MAE): {np.mean(loso_results):.3f}")
    print(f" >>> Standard Deviation: {np.std(loso_results):.3f}")
    print("-"*80)

    # plot_folds_test_subplot(
    #     viz_cache,
    #     fs=CONFIG["fs"],
    #     title="Fold-wise TEST visualization (Pred by window-avg rate)"
    # )

    # for item in viz_cache:
    #     plot_phase_heatmap_and_dominant(
    #         item["phase_p"],
    #         fs=CONFIG["fs"],
    #         title=f"[Fold {item['fold']:2d}] {item['test_subj']} | k_hat(full)={item['k_hat']:.2f} | ent(full)={item['entropy']:.3f}",
    #         max_T=2000
    #     )


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

--------------------------------------------------------------------------------
 >>> Starting LOSO (count-only, K-auto) + WINDOWING
--------------------------------------------------------------------------------
Fold  1 | Test: subject1 | MAE: 0.42 | [Pred(win): 20.4 / GT: 20]
[Fold TEST Summary] subject1 | GT=20 | Pred(win)=20.42 | Diff=+0.42 | k_hat(full)=2.68 | phase_entropy(full)=0.880
Fold  2 | Test: subject2 | MAE: 0.30 | [Pred(win): 22.3 / GT: 22]
[Fold TEST Summary] subject2 | GT=22 | Pred(win)=22.30 | Diff=+0.30 | k_hat(full)=1.73 | phase_entropy(full)=0.663
Fold  3 | Test: subject3 | MAE: 0.33 | [Pred(win): 20.7 / GT: 21]
[Fold TEST Summary] subject3 | GT=21 | Pred(win)=20.67 | Diff=-0.33 | k_hat(full)=2.24 | phase_entropy(full)=0.783
Fold  4 | Test: subject4 | MAE: 2.14 | [Pred(win): 23.1 / GT: 21]
[Fold TEST Summary] subject4 | GT=21 | Pred(win)=23.14 | Diff=+2.14 | k_

In [6]:
# =========================
# Count-only K-auto (Multi-event) + Windowing version
#
# ✅ Windowing added:
# - TRAIN: trial -> sliding windows (window-level count = trial-average rate * window duration)
# - TEST : trial 그대로 두고, windowing inference로 window rate 평균 -> 전체 count 예측
# - k_hat / entropy / rep_rate / phase heatmap은 (표현학습 확인용) full-trial 1회 forward로 기록
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (표준화) 평균=0, std=1
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T, C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) ✅ Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows 확장
    window 라벨은 trial-level 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> sliding windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (이미 정규화된 상태)
    return: pred_count(float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)  # (B,T,K), sum=1
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    """
    - outputs K_max micro-event rates r_k(t)
    - predicts k_hat (>=1) per sample
    - rep_rate(t) = micro_rate(t) / k_hat
    """
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp logit bias만 -2

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        # ✅ BUGFIX: use tau argument (was tau=1.0 fixed)
        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p  # (B,T,K)

        micro_rate_t = amp_t  # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,
            "phase_p": phase_p,
            "phase_logits": phase_logits,
            "micro_rate_t": micro_rate_t,
            "rep_rate_t": rep_rate_t,
            "k_hat": k_hat,
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps  # valid(B*T)*C
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])  # (B,T-1)
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)  # (B,K)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)  # (B,T,1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)         # (B,C,T)
        mask = batch["mask"].to(device)      # (B,T)
        y_count = batch["count"].to(device)  # (B,)
        length = batch["length"].to(device)  # (B,)

        duration = torch.clamp(length / fs, min=1e-6)  # sec
        y_rate = y_count / duration                    # reps/s

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = len(loader)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers (subject-wise subplot)
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)  # (T,)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(
    phase_p_np,
    fs,
    title="phase_p heatmap + dominant phase",
    max_T=2000
):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)  # (T',K)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)  # (T',)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :],
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization (only test_subj)"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred(win) {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main (LOSO)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            4: 'Walking',
            10: 'Jogging',
            11: 'Running',

        },
        "ACT_FEATURE_MAP": {
            4: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            10: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            11: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # ✅ Windowing Params (added)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        # temperature
        "tau": 1.0,

        # Count-only labels
        "ALL_LABELS": [
            ("subject1", 4, 113),
            ("subject2", 4, 99),
            ("subject3", 4, 104),
            ("subject4", 4, 112),
            ("subject5", 4, 109),
            ("subject6", 4, 111),
            ("subject7", 4, 106),
            ("subject8", 4, 95),
            ("subject9", 4, 111),
            ("subject10", 4, 102),
        ],
    }

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    loso_results = []

    print("\n" + "-"*80)
    print(" >>> Starting LOSO (count-only, K-auto) + WINDOWING")
    print("-"*80)

    viz_cache = []

    for fold_idx, test_subj in enumerate(subjects):
        set_strict_seed(CONFIG["seed"])

        train_labels = [x for x in CONFIG["ALL_LABELS"] if x[0] != test_subj]
        test_labels  = [x for x in CONFIG["ALL_LABELS"] if x[0] == test_subj]

        # ---- trial-level ----
        train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if not test_trials:
            print(f"[Skip] Fold {fold_idx+1}: {test_subj} has no data.")
            continue

        # ✅ TRAIN only: trial -> windows
        train_data = trial_list_to_windows(
            train_trials,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            drop_last=CONFIG["drop_last"]
        )

        # TEST: trial 그대로
        test_data = test_trials

        g = torch.Generator()
        g.manual_seed(CONFIG["seed"])

        train_loader = DataLoader(
            TrialDataset(train_data),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            generator=g,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        model.eval()

        # ---- fold test (trial 단위) ----
        fold_mae = 0.0
        fold_res_str = ""

        test_gt = None
        test_pred = None
        test_diff = None
        test_khat = None
        test_entropy = None

        test_t = None
        test_rep_rate = None
        test_phase_p = None

        for item in test_data:
            x_np = item["data"]  # (T,C)
            T = x_np.shape[0]

            # ✅ pred: windowing inference
            count_pred_win, _win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )

            count_gt = float(item["count"])
            abs_err = abs(count_pred_win - count_gt)
            fold_mae += abs_err
            fold_res_str += f"[Pred(win): {count_pred_win:.1f} / GT: {count_gt:.0f}]"

            # ✅ 표현학습 확인용: full-trial 1회 forward (k_hat/entropy/rep_rate/phase)
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            test_gt = count_gt
            test_pred = float(count_pred_win)
            test_diff = float(count_pred_win - count_gt)
            test_khat = k_hat
            test_entropy = ent

            rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()  # (T,)
            test_rep_rate = rep_rate
            test_t = np.arange(T) / CONFIG["fs"]
            test_phase_p = phase_p

        fold_mae /= len(test_data)
        loso_results.append(fold_mae)

        print(f"Fold {fold_idx+1:2d} | Test: {test_subj} | MAE: {fold_mae:.2f} | {fold_res_str}")

        if (test_gt is not None) and (test_pred is not None):
            print(
                f"[Fold TEST Summary] {test_subj} | GT={test_gt:.0f} | Pred(win)={test_pred:.2f} | "
                f"Diff={test_diff:+.2f} | k_hat(full)={test_khat:.2f} | phase_entropy(full)={test_entropy:.3f}"
            )

        if (test_t is not None) and (test_rep_rate is not None):
            viz_cache.append({
                "fold": fold_idx + 1,
                "test_subj": test_subj,
                "t": test_t,
                "rep_rate": test_rep_rate,
                "gt": float(test_gt) if test_gt is not None else 0.0,
                "pred": float(test_pred) if test_pred is not None else 0.0,
                "diff": float(test_diff) if test_diff is not None else 0.0,
                "k_hat": float(test_khat) if test_khat is not None else 0.0,
                "entropy": float(test_entropy) if test_entropy is not None else 0.0,
                "phase_p": test_phase_p,
            })

    print("-"*80)
    print(f" >>> Final LOSO Result (Average MAE): {np.mean(loso_results):.3f}")
    print(f" >>> Standard Deviation: {np.std(loso_results):.3f}")
    print("-"*80)

    # plot_folds_test_subplot(
    #     viz_cache,
    #     fs=CONFIG["fs"],
    #     title="Fold-wise TEST visualization (Pred by window-avg rate)"
    # )

    # for item in viz_cache:
    #     plot_phase_heatmap_and_dominant(
    #         item["phase_p"],
    #         fs=CONFIG["fs"],
    #         title=f"[Fold {item['fold']:2d}] {item['test_subj']} | k_hat(full)={item['k_hat']:.2f} | ent(full)={item['entropy']:.3f}",
    #         max_T=2000
    #     )


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

--------------------------------------------------------------------------------
 >>> Starting LOSO (count-only, K-auto) + WINDOWING
--------------------------------------------------------------------------------
Fold  1 | Test: subject1 | MAE: 15.00 | [Pred(win): 98.0 / GT: 113]
[Fold TEST Summary] subject1 | GT=113 | Pred(win)=98.00 | Diff=-15.00 | k_hat(full)=2.36 | phase_entropy(full)=0.627
Fold  2 | Test: subject2 | MAE: 10.34 | [Pred(win): 109.3 / GT: 99]
[Fold TEST Summary] subject2 | GT=99 | Pred(win)=109.34 | Diff=+10.34 | k_hat(full)=2.36 | phase_entropy(full)=0.660
Fold  3 | Test: subject3 | MAE: 6.00 | [Pred(win): 110.0 / GT: 104]
[Fold TEST Summary] subject3 | GT=104 | Pred(win)=110.00 | Diff=+6.00 | k_hat(full)=2.21 | phase_entropy(full)=0.633
Fold  4 | Test: subject4 | MAE: 1.85 | [Pred(win): 110.1 / GT: 112]
[Fold TEST Summary] subject4 | GT=112 | Pred(win)=110.15 |

In [7]:
# =========================
# Count-only K-auto (Multi-event) + Windowing version
#
# ✅ Windowing added:
# - TRAIN: trial -> sliding windows (window-level count = trial-average rate * window duration)
# - TEST : trial 그대로 두고, windowing inference로 window rate 평균 -> 전체 count 예측
# - k_hat / entropy / rep_rate / phase heatmap은 (표현학습 확인용) full-trial 1회 forward로 기록
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (표준화) 평균=0, std=1
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T, C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) ✅ Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows 확장
    window 라벨은 trial-level 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> sliding windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (이미 정규화된 상태)
    return: pred_count(float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)  # (B,T,K), sum=1
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    """
    - outputs K_max micro-event rates r_k(t)
    - predicts k_hat (>=1) per sample
    - rep_rate(t) = micro_rate(t) / k_hat
    """
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp logit bias만 -2

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        # ✅ BUGFIX: use tau argument (was tau=1.0 fixed)
        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p  # (B,T,K)

        micro_rate_t = amp_t  # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,
            "phase_p": phase_p,
            "phase_logits": phase_logits,
            "micro_rate_t": micro_rate_t,
            "rep_rate_t": rep_rate_t,
            "k_hat": k_hat,
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps  # valid(B*T)*C
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])  # (B,T-1)
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)  # (B,K)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)  # (B,T,1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)         # (B,C,T)
        mask = batch["mask"].to(device)      # (B,T)
        y_count = batch["count"].to(device)  # (B,)
        length = batch["length"].to(device)  # (B,)

        duration = torch.clamp(length / fs, min=1e-6)  # sec
        y_rate = y_count / duration                    # reps/s

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = len(loader)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers (subject-wise subplot)
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)  # (T,)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(
    phase_p_np,
    fs,
    title="phase_p heatmap + dominant phase",
    max_T=2000
):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)  # (T',K)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)  # (T',)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :],
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization (only test_subj)"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred(win) {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main (LOSO)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            4: 'Walking',
            10: 'Jogging',
            11: 'Running',

        },
        "ACT_FEATURE_MAP": {
            4: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            10: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            11: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # ✅ Windowing Params (added)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        # temperature
        "tau": 1.0,

        # Count-only labels
        "ALL_LABELS": [
            ("subject1", 10, 157),
            ("subject2", 10, 161),
            ("subject3", 10, 154),
            ("subject4", 10, 154),
            ("subject5", 10, 160),
            ("subject6", 10, 156),
            ("subject7", 10, 153),
            ("subject8", 10, 160),
            ("subject9", 10, 166),
            ("subject10", 10, 156),
        ],
    }

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    loso_results = []

    print("\n" + "-"*80)
    print(" >>> Starting LOSO (count-only, K-auto) + WINDOWING")
    print("-"*80)

    viz_cache = []

    for fold_idx, test_subj in enumerate(subjects):
        set_strict_seed(CONFIG["seed"])

        train_labels = [x for x in CONFIG["ALL_LABELS"] if x[0] != test_subj]
        test_labels  = [x for x in CONFIG["ALL_LABELS"] if x[0] == test_subj]

        # ---- trial-level ----
        train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if not test_trials:
            print(f"[Skip] Fold {fold_idx+1}: {test_subj} has no data.")
            continue

        # ✅ TRAIN only: trial -> windows
        train_data = trial_list_to_windows(
            train_trials,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            drop_last=CONFIG["drop_last"]
        )

        # TEST: trial 그대로
        test_data = test_trials

        g = torch.Generator()
        g.manual_seed(CONFIG["seed"])

        train_loader = DataLoader(
            TrialDataset(train_data),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            generator=g,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        model.eval()

        # ---- fold test (trial 단위) ----
        fold_mae = 0.0
        fold_res_str = ""

        test_gt = None
        test_pred = None
        test_diff = None
        test_khat = None
        test_entropy = None

        test_t = None
        test_rep_rate = None
        test_phase_p = None

        for item in test_data:
            x_np = item["data"]  # (T,C)
            T = x_np.shape[0]

            # ✅ pred: windowing inference
            count_pred_win, _win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )

            count_gt = float(item["count"])
            abs_err = abs(count_pred_win - count_gt)
            fold_mae += abs_err
            fold_res_str += f"[Pred(win): {count_pred_win:.1f} / GT: {count_gt:.0f}]"

            # ✅ 표현학습 확인용: full-trial 1회 forward (k_hat/entropy/rep_rate/phase)
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            test_gt = count_gt
            test_pred = float(count_pred_win)
            test_diff = float(count_pred_win - count_gt)
            test_khat = k_hat
            test_entropy = ent

            rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()  # (T,)
            test_rep_rate = rep_rate
            test_t = np.arange(T) / CONFIG["fs"]
            test_phase_p = phase_p

        fold_mae /= len(test_data)
        loso_results.append(fold_mae)

        print(f"Fold {fold_idx+1:2d} | Test: {test_subj} | MAE: {fold_mae:.2f} | {fold_res_str}")

        if (test_gt is not None) and (test_pred is not None):
            print(
                f"[Fold TEST Summary] {test_subj} | GT={test_gt:.0f} | Pred(win)={test_pred:.2f} | "
                f"Diff={test_diff:+.2f} | k_hat(full)={test_khat:.2f} | phase_entropy(full)={test_entropy:.3f}"
            )

        if (test_t is not None) and (test_rep_rate is not None):
            viz_cache.append({
                "fold": fold_idx + 1,
                "test_subj": test_subj,
                "t": test_t,
                "rep_rate": test_rep_rate,
                "gt": float(test_gt) if test_gt is not None else 0.0,
                "pred": float(test_pred) if test_pred is not None else 0.0,
                "diff": float(test_diff) if test_diff is not None else 0.0,
                "k_hat": float(test_khat) if test_khat is not None else 0.0,
                "entropy": float(test_entropy) if test_entropy is not None else 0.0,
                "phase_p": test_phase_p,
            })

    print("-"*80)
    print(f" >>> Final LOSO Result (Average MAE): {np.mean(loso_results):.3f}")
    print(f" >>> Standard Deviation: {np.std(loso_results):.3f}")
    print("-"*80)

    # plot_folds_test_subplot(
    #     viz_cache,
    #     fs=CONFIG["fs"],
    #     title="Fold-wise TEST visualization (Pred by window-avg rate)"
    # )

    # for item in viz_cache:
    #     plot_phase_heatmap_and_dominant(
    #         item["phase_p"],
    #         fs=CONFIG["fs"],
    #         title=f"[Fold {item['fold']:2d}] {item['test_subj']} | k_hat(full)={item['k_hat']:.2f} | ent(full)={item['entropy']:.3f}",
    #         max_T=2000
    #     )


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

--------------------------------------------------------------------------------
 >>> Starting LOSO (count-only, K-auto) + WINDOWING
--------------------------------------------------------------------------------
Fold  1 | Test: subject1 | MAE: 3.03 | [Pred(win): 154.0 / GT: 157]
[Fold TEST Summary] subject1 | GT=157 | Pred(win)=153.97 | Diff=-3.03 | k_hat(full)=1.29 | phase_entropy(full)=0.365
Fold  2 | Test: subject2 | MAE: 5.05 | [Pred(win): 166.0 / GT: 161]
[Fold TEST Summary] subject2 | GT=161 | Pred(win)=166.05 | Diff=+5.05 | k_hat(full)=1.74 | phase_entropy(full)=0.504
Fold  3 | Test: subject3 | MAE: 6.57 | [Pred(win): 147.4 / GT: 154]
[Fold TEST Summary] subject3 | GT=154 | Pred(win)=147.43 | Diff=-6.57 | k_hat(full)=1.47 | phase_entropy(full)=0.345
Fold  4 | Test: subject4 | MAE: 4.35 | [Pred(win): 158.3 / GT: 154]
[Fold TEST Summary] subject4 | GT=154 | Pred(win)=158.35 |

In [8]:
# =========================
# Count-only K-auto (Multi-event) + Windowing version
#
# ✅ Windowing added:
# - TRAIN: trial -> sliding windows (window-level count = trial-average rate * window duration)
# - TEST : trial 그대로 두고, windowing inference로 window rate 평균 -> 전체 count 예측
# - k_hat / entropy / rep_rate / phase heatmap은 (표현학습 확인용) full-trial 1회 forward로 기록
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (표준화) 평균=0, std=1
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,              # (T, C)
                'count': float(gt_count),      # trial total count
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) ✅ Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=8.0, stride_sec=4.0, drop_last=True):
    """
    TRAIN 전용: trial -> sliding windows 확장
    window 라벨은 trial-level 평균 rate로부터 생성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]  # (T,C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur  # reps/s

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    TEST 전용: trial -> sliding windows inference -> window rate 평균 -> total count
    x_np: (T,C) numpy (이미 정규화된 상태)
    return: pred_count(float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    # short trial -> 1회 forward
    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


# ---------------------------------------------------------------------
# 2.8) Dataset / Collate
# ---------------------------------------------------------------------
class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)  # (B,T,K), sum=1
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    """
    - outputs K_max micro-event rates r_k(t)
    - predicts k_hat (>=1) per sample
    - rep_rate(t) = micro_rate(t) / k_hat
    """
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)  # amp logit bias만 -2

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        # ✅ BUGFIX: use tau argument (was tau=1.0 fixed)
        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p  # (B,T,K)

        micro_rate_t = amp_t  # (B,T)

        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6) # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,
            "phase_p": phase_p,
            "phase_logits": phase_logits,
            "micro_rate_t": micro_rate_t,
            "rep_rate_t": rep_rate_t,
            "k_hat": k_hat,
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)              # (B,1,T)
    se = (x_hat - x) ** 2                    # (B,C,T)
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps  # valid(B*T)*C
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])  # (B,T-1)
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)  # (B,T)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)  # (B,K)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)  # (B,T,1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)         # (B,C,T)
        mask = batch["mask"].to(device)      # (B,T)
        y_count = batch["count"].to(device)  # (B,)
        length = batch["length"].to(device)  # (B,)

        duration = torch.clamp(length / fs, min=1e-6)  # sec
        y_rate = y_count / duration                    # reps/s

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = len(loader)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers (subject-wise subplot)
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)  # (T,)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(
    phase_p_np,
    fs,
    title="phase_p heatmap + dominant phase",
    max_T=2000
):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)  # (T',K)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)  # (T',)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T,
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :],
        aspect="auto",
        origin="lower",
        interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization (only test_subj)"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred(win) {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat(full)={k_hat:.2f} | phase_entropy(full)={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main (LOSO)
# ---------------------------------------------------------------------
def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],
        "TARGET_ACTIVITIES_MAP": {
            4: 'Walking',
            10: 'Jogging',
            11: 'Running',

        },
        "ACT_FEATURE_MAP": {
            4: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            10: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
            11: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
                'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z'
                ],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # ✅ Windowing Params (added)
        "win_sec": 8.0,
        "stride_sec": 4.0,
        "drop_last": True,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.0075,

        # temperature
        "tau": 1.0,

        # Count-only labels
        "ALL_LABELS": [
            ("subject1", 11, 165),
            ("subject2", 11, 158),
            ("subject3", 11, 174),
            ("subject4", 11, 163),
            ("subject5", 11, 157),
            ("subject6", 11, 172),
            ("subject7", 11, 149),
            ("subject8", 11, 166),
            ("subject9", 11, 174),
            ("subject10", 11, 172),
        ],
    }

    set_strict_seed(CONFIG["seed"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(CONFIG["data_dir"], CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["COLUMN_NAMES"])
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]
    loso_results = []

    print("\n" + "-"*80)
    print(" >>> Starting LOSO (count-only, K-auto) + WINDOWING")
    print("-"*80)

    viz_cache = []

    for fold_idx, test_subj in enumerate(subjects):
        set_strict_seed(CONFIG["seed"])

        train_labels = [x for x in CONFIG["ALL_LABELS"] if x[0] != test_subj]
        test_labels  = [x for x in CONFIG["ALL_LABELS"] if x[0] == test_subj]

        # ---- trial-level ----
        train_trials = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_trials  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if not test_trials:
            print(f"[Skip] Fold {fold_idx+1}: {test_subj} has no data.")
            continue

        # ✅ TRAIN only: trial -> windows
        train_data = trial_list_to_windows(
            train_trials,
            fs=CONFIG["fs"],
            win_sec=CONFIG["win_sec"],
            stride_sec=CONFIG["stride_sec"],
            drop_last=CONFIG["drop_last"]
        )

        # TEST: trial 그대로
        test_data = test_trials

        g = torch.Generator()
        g.manual_seed(CONFIG["seed"])

        train_loader = DataLoader(
            TrialDataset(train_data),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            generator=g,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        model.eval()

        # ---- fold test (trial 단위) ----
        fold_mae = 0.0
        fold_res_str = ""

        test_gt = None
        test_pred = None
        test_diff = None
        test_khat = None
        test_entropy = None

        test_t = None
        test_rep_rate = None
        test_phase_p = None

        for item in test_data:
            x_np = item["data"]  # (T,C)
            T = x_np.shape[0]

            # ✅ pred: windowing inference
            count_pred_win, _win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["win_sec"],
                stride_sec=CONFIG["stride_sec"],
                device=device,
                tau=CONFIG.get("tau", 1.0),
                batch_size=CONFIG.get("batch_size", 64)
            )

            count_gt = float(item["count"])
            abs_err = abs(count_pred_win - count_gt)
            fold_mae += abs_err
            fold_res_str += f"[Pred(win): {count_pred_win:.1f} / GT: {count_gt:.0f}]"

            # ✅ 표현학습 확인용: full-trial 1회 forward (k_hat/entropy/rep_rate/phase)
            x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
            with torch.no_grad():
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG.get("tau", 1.0))

            phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()  # (T,K)
            k_hat = float(aux["k_hat"].item())
            ent = compute_phase_entropy_mean(phase_p)

            test_gt = count_gt
            test_pred = float(count_pred_win)
            test_diff = float(count_pred_win - count_gt)
            test_khat = k_hat
            test_entropy = ent

            rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()  # (T,)
            test_rep_rate = rep_rate
            test_t = np.arange(T) / CONFIG["fs"]
            test_phase_p = phase_p

        fold_mae /= len(test_data)
        loso_results.append(fold_mae)

        print(f"Fold {fold_idx+1:2d} | Test: {test_subj} | MAE: {fold_mae:.2f} | {fold_res_str}")

        if (test_gt is not None) and (test_pred is not None):
            print(
                f"[Fold TEST Summary] {test_subj} | GT={test_gt:.0f} | Pred(win)={test_pred:.2f} | "
                f"Diff={test_diff:+.2f} | k_hat(full)={test_khat:.2f} | phase_entropy(full)={test_entropy:.3f}"
            )

        if (test_t is not None) and (test_rep_rate is not None):
            viz_cache.append({
                "fold": fold_idx + 1,
                "test_subj": test_subj,
                "t": test_t,
                "rep_rate": test_rep_rate,
                "gt": float(test_gt) if test_gt is not None else 0.0,
                "pred": float(test_pred) if test_pred is not None else 0.0,
                "diff": float(test_diff) if test_diff is not None else 0.0,
                "k_hat": float(test_khat) if test_khat is not None else 0.0,
                "entropy": float(test_entropy) if test_entropy is not None else 0.0,
                "phase_p": test_phase_p,
            })

    print("-"*80)
    print(f" >>> Final LOSO Result (Average MAE): {np.mean(loso_results):.3f}")
    print(f" >>> Standard Deviation: {np.std(loso_results):.3f}")
    print("-"*80)

    # plot_folds_test_subplot(
    #     viz_cache,
    #     fs=CONFIG["fs"],
    #     title="Fold-wise TEST visualization (Pred by window-avg rate)"
    # )

    # for item in viz_cache:
    #     plot_phase_heatmap_and_dominant(
    #         item["phase_p"],
    #         fs=CONFIG["fs"],
    #         title=f"[Fold {item['fold']:2d}] {item['test_subj']} | k_hat(full)={item['k_hat']:.2f} | ent(full)={item['entropy']:.3f}",
    #         max_T=2000
    #     )


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

--------------------------------------------------------------------------------
 >>> Starting LOSO (count-only, K-auto) + WINDOWING
--------------------------------------------------------------------------------
Fold  1 | Test: subject1 | MAE: 2.92 | [Pred(win): 162.1 / GT: 165]
[Fold TEST Summary] subject1 | GT=165 | Pred(win)=162.08 | Diff=-2.92 | k_hat(full)=1.62 | phase_entropy(full)=0.315
Fold  2 | Test: subject2 | MAE: 14.48 | [Pred(win): 172.5 / GT: 158]
[Fold TEST Summary] subject2 | GT=158 | Pred(win)=172.48 | Diff=+14.48 | k_hat(full)=1.98 | phase_entropy(full)=0.317
Fold  3 | Test: subject3 | MAE: 21.68 | [Pred(win): 152.3 / GT: 174]
[Fold TEST Summary] subject3 | GT=174 | Pred(win)=152.32 | Diff=-21.68 | k_hat(full)=1.35 | phase_entropy(full)=0.309
Fold  4 | Test: subject4 | MAE: 19.88 | [Pred(win): 182.9 / GT: 163]
[Fold TEST Summary] subject4 | GT=163 | Pred(win)=182